Comparing bonus unemployment, payroll tax cuts, and universal payments
=============================================================

This uses the 2018 Current Population Survey March Supplement, Tax-Calculator, and the Supplemental Poverty Measure to estimate the effects of the Federal Pandemic Unemployment Compensation (extra 600 dollars per week), a budget-neutral payroll tax cuts, and a budget-neutral universal payment.

In [1]:
import numpy as np
import pandas as pd
import microdf as mdf
import plotly.express as px
import plotly

spmu = pd.read_csv('spmu.csv.gz')
person = pd.read_csv('person.csv.gz')

# All potential reforms.
CHG_COLS = ['fpuc_net', 'fpuc_ubi', 'fpuc_adult_ubi', 'fpuc_fica_cut',
            'fpuc2_net', 'fpuc2_ubi', 'fpuc2_adult_ubi', 'fpuc2_fica_cut']

## Poverty analysis

In [2]:
def pov(reform, year, age_group='All', race='All'):
    """ Calculate the poverty rate under the specified reform for the
        specified population.
        Note: All arguments refer to the poverty population, not the reform.
    
    Args:
        reform: One of CHG_COLS. If None, provides the baseline rate.
        year: Year of the data (year before CPS survey year).
        age_group: Age group, either
            - 'Children' (under 18)
            - 'Adults' (18 or over)
            - 'All'
        race: Race code to filter to. Defaults to 'All'.
        
    Returns:
        2018 SPM poverty rate.
    """
    if reform == 'baseline':
        resource_col = 'spmpoor'
    else:
        resource_col = 'spmpoor_' + reform
    target_persons = person[person.FLPDYR == year]
    if age_group == 'Children':
        target_persons = target_persons[target_persons.age < 18]
    elif age_group == 'Adults':
        target_persons = target_persons[target_persons.age >= 18]
    if race != 'All':
        target_persons = target_persons[target_persons.race == race]
    return mdf.weighted_mean(target_persons, resource_col, 'asecwt')

def pov_row(row):
    """ Calculate poverty based on parameters specified in the row.
    
    Args:
        row: pandas Series.
        
    Returns:
        2018 SPM poverty rate.
    """
    return pov(row.reform, row.age_group, row.race)

In [3]:
pov_rates = mdf.cartesian_product({'reform': ['baseline'] + CHG_COLS,
                                   'year': person.FLPDYR.unique(),
                                   'age_group': ['All', 'Children', 'Adults'],
                                   'race': ['All', 200]})  # 200 means Black.
pov_rates['pov'] = 100 * pov_rates.apply(pov_row, axis=1)

/home/mghenis/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/home/mghenis/anaconda3/lib/python3.7/site-packages/microdf/weighted.py:45: RuntimeWarning: invalid value encountered in double_scalars
  return weighted_sum(df, col, w) / df[w].sum()


Top-level poverty rates.

In [4]:
pov_rates[(pov_rates.age_group == 'All') & (pov_rates.race == 'All')]

,reform,year,age_group,race,pov
0,baseline,2009,All,All,NaN
6,baseline,2013,All,All,NaN
12,baseline,2010,All,All,NaN
18,baseline,2014,All,All,NaN
24,baseline,2015,All,All,NaN
...,...,...,...,...,...
510,fpuc2_fica_cut,2017,All,All,NaN
516,fpuc2_fica_cut,2011,All,All,NaN
522,fpuc2_fica_cut,2012,All,All,NaN
528,fpuc2_fica_cut,2016,All,All,NaN


Top-level child poverty rates.

In [5]:
pov_rates[(pov_rates.age_group == 'Children') & (pov_rates.race == 'All')]

,reform,year,age_group,race,pov
2,baseline,2009,Children,All,NaN
8,baseline,2013,Children,All,NaN
14,baseline,2010,Children,All,NaN
20,baseline,2014,Children,All,NaN
26,baseline,2015,Children,All,NaN
...,...,...,...,...,...
512,fpuc2_fica_cut,2017,Children,All,NaN
518,fpuc2_fica_cut,2011,Children,All,NaN
524,fpuc2_fica_cut,2012,Children,All,NaN
530,fpuc2_fica_cut,2016,Children,All,NaN


### Poverty gap and inequality

Calculate these for all people and SPM units, without breaking out by age or race.

In [8]:
def pov_gap_b(reform, year):
    if reform == 'baseline':
        resource_col = 'spmtotres'
    else:
        resource_col = 'spmtotres_' + reform
    tmp = spmu[spmu.FLPDYR == year]
    pov_gap = np.maximum(tmp.spmthresh - tmp[resource_col], 0)
    return (pov_gap * tmp.spmwt).sum() / 1e9

def pov_gap_row(row):
    return pov_gap_b(row.reform, row.year)

In [9]:
pov_gap_ineq = pov_rates[['reform', 'year']].drop_duplicates()
pov_gap_ineq['pov_gap_b'] = pov_gap_ineq.apply(pov_gap_row, axis=1)

### Inequality

By individual based on their percentage of SPM resources.

In [10]:
def gini(reform, year):
    if reform == 'baseline':
        resource_col = 'spmtotres'
    else:
        resource_col = 'spmtotres_' + reform
    tmp = person[person.FLPDYR == year]
    return mdf.gini(tmp[resource_col], tmp.asecwt)

def gini_row(row):
    return gini(row.reform, row.year)

In [11]:
pov_gap_ineq['gini'] = pov_gap_ineq.apply(gini_row, axis=1)
pov_gap_ineq

,reform,year,pov_gap_b,gini
0,baseline,2009,161.105442,0.409077
6,baseline,2013,355.410775,0.424085
12,baseline,2010,174.763681,0.409032
18,baseline,2014,177.933852,0.423383
24,baseline,2015,176.353623,0.422258
...,...,...,...,...
510,fpuc2_fica_cut,2017,143.934232,0.471596
516,fpuc2_fica_cut,2011,118.644339,0.473194
522,fpuc2_fica_cut,2012,115.753031,0.472583
528,fpuc2_fica_cut,2016,129.504776,0.471651
